# Decision Trees 
- Problem : Titanic Survior Prediction Kaggle Challenge
## Learning Goals
- How to preprocess data ?
    - Dropping not useful features
    - Filling the missing values(Data Impultation)
- Creating a Binary Decision Tree from Scratch

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('titanic.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# to see all the no. of columns

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
columns_to_drop = ["PassengerId","Name","Ticket","Cabin","Embarked"]

In [6]:
data_clean = data.drop(columns_to_drop,axis =1)

In [7]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data_clean["Sex"] = le.fit_transform(data_clean["Sex"])

In [9]:
data_clean = data_clean.fillna(data_clean["Age"].mean())

In [10]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(5)
memory usage: 48.8 KB


In [11]:
from sklearn.preprocessing.imputation import Imputer


In [12]:
impu = Imputer

In [13]:
# loc is function in pandas which allows to see all the values on that perticular row
data_clean.loc[1]

Survived     1.0000
Pclass       1.0000
Sex          0.0000
Age         38.0000
SibSp        1.0000
Parch        0.0000
Fare        71.2833
Name: 1, dtype: float64

In [14]:
i = ["Pclass","Sex","Age","SibSp","Parch","Fare"]

X = data_clean[i]
Y = data_clean["Survived"]

In [15]:
def entropy(col):
    
    counts = np.unique(col,return_counts=True)
    
    N = float(col.shape[0])
    
    ent = 0.0
    
    for ix in counts[1]:
        p = ix/N
        ent += (-1.0*p*np.log2(p)) 
        
    return ent

In [16]:
e = np.array(['y','y','y','n','n'])
a = entropy(e)
print(a)

0.9709505944546686


In [17]:
def divide_data(x_data,fkey,fval):
    # Constructing a binary tree by spliting up the data on the basis of one column which gives us high information
    # gain (i.e. fkey) and comparing each value in this column with the avg. of the column (i.e. fval)
    
    # Creating two dataframes for left and right 
    x_right = pd.DataFrame([],columns = x_data.columns)
    x_left = pd.DataFrame([],columns = x_data.columns)
    
    # iterating over all the values of the column(i.e. fkey)
    for ix in range (x_data.shape[0]):
        
        val = x_data[fkey].loc[ix]
        
        if val > fval:
            
            x_right = x_right.append(x_data.loc[ix])
        else :
            x_left = x_left.append(x_data.loc[ix])
    
    return x_left,x_right

In [18]:
def information_gain(x_data,fkey,fval):
    
    left,right = divide_data(x_data,fkey,fval)
    
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    # All examples come to one side
    
    if left.shape[0] == 0 or right.shape[0] ==0:
        
        return -100000000 # Min information gain that's why we return -infinity
    
    i_gain = entropy(x_data.Survived) - (l*entropy(left.Survived) + r*entropy(right.Survived))
    
    return i_gain

In [19]:
# Test our function 

for fx in X.columns:
    print(fx)
    print(information_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.0008836151229467681
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [20]:
class DecisionTree:
    
    def __init__(self,depth=0,max_depth=5):
        
        self.left = None
        self.right = None
        self.fkey = None  # Its the node name 
        self.fval = None
        self.max_depth = max_depth 
        self.depth = depth
        self.target = None
    def train(self,X_train):
        features = ['Pclass','Sex','Age','SibSp','Parch','Fare']
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
        
        self.fkey = features[np.argmax(info_gains)]
        self.fval = X_train[self.fkey].mean()
        
        print("Making Tree Feature is ",self.fkey)
        
        
        #Split Data
        
        data_left,data_right = divide_data(X_train,self.fkey,self.fval)
        data_left = data_left.reset_index(drop =True)
        data_right = data_right.reset_index(drop = True)
        
        #Truly a left node
        if data_left.shape[0] ==0 or data_right.shape[0] ==0:
            
            if X_train.Survived.mean() >= 0.5:
                self.target = 'Survive'
            else:
                self.target = 'Dead'
            return
        
        # Stop early when depth >= max depth
        if(self.depth >= self.max_depth):
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target ="Dead"
            return
        
        # Recursive Case
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth +1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        # You can set the target at every node
        if X_train.Survived.mean() >=0.5:
            self.target = "Survive"
        else:
            self.target ="Dead"
        return
    
    def predict(self,test):
        
        if test[self.fkey] > self.fval:
            #goto Right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            
            if self.left is None:
                return self.target
            return self.left.predict(test)
        
            
    
    
    
            
    
        
            
        

In [21]:
#d = DecisionTree()
#d.train(data_clean)

## Train-Validation-Test Set Split

In [22]:
split = int(0.7*data_clean.shape[0])
train_data = data_clean[:split]
test_data = data_clean[split:]
test_data = test_data.reset_index(drop = True)

In [23]:
print(train_data.shape,test_data.shape)

(623, 7) (268, 7)


In [24]:
dt = DecisionTree()

In [25]:
dt.train(train_data)

Making Tree Feature is  Sex
Making Tree Feature is  Pclass
Making Tree Feature is  Age
Making Tree Feature is  SibSp
Making Tree Feature is  Pclass
Making Tree Feature is  Age
Making Tree Feature is  Age
Making Tree Feature is  SibSp
Making Tree Feature is  Parch
Making Tree Feature is  Pclass
Making Tree Feature is  SibSp
Making Tree Feature is  Fare
Making Tree Feature is  Parch
Making Tree Feature is  Age
Making Tree Feature is  Pclass
Making Tree Feature is  Age
Making Tree Feature is  Age
Making Tree Feature is  Parch
Making Tree Feature is  SibSp
Making Tree Feature is  Fare
Making Tree Feature is  Age
Making Tree Feature is  Age
Making Tree Feature is  Fare
Making Tree Feature is  Age
Making Tree Feature is  Age
Making Tree Feature is  Fare
Making Tree Feature is  Age
Making Tree Feature is  Parch
Making Tree Feature is  Fare
Making Tree Feature is  Fare
Making Tree Feature is  Fare
Making Tree Feature is  Age
Making Tree Feature is  Fare
Making Tree Feature is  Parch
Making Tre

In [26]:
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
print(dt.right.fkey)

Sex
0.6292134831460674
Pclass
Fare


In [27]:
y_pred = []

for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

In [28]:
y_pred

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'De

In [29]:
y_actual = test_data['Survived']
print(y_actual)


0      0
1      0
2      0
3      0
4      1
5      0
6      0
7      1
8      0
9      1
10     0
11     0
12     1
13     0
14     0
15     0
16     0
17     0
18     1
19     0
20     1
21     1
22     1
23     0
24     1
25     0
26     1
27     0
28     1
29     0
      ..
238    0
239    1
240    0
241    0
242    1
243    1
244    0
245    0
246    1
247    0
248    1
249    0
250    0
251    1
252    1
253    0
254    0
255    0
256    1
257    1
258    0
259    0
260    0
261    0
262    0
263    0
264    1
265    0
266    1
267    0
Name: Survived, Length: 268, dtype: int64


In [30]:
le = LabelEncoder()
y_pred = le.fit_transform(y_pred)

In [31]:
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0
 0 0 0 0 0 1 0 0 0]


In [32]:

print(y_pred.shape)
print(y_actual.shape)

(268,)
(268,)


In [33]:
acc = np.sum(y_pred == y_actual)/y_pred.shape[0]
print(acc)

0.8171641791044776


# Decision Tree using SkLearn

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
sk_tree = DecisionTreeClassifier(criterion = 'entropy',max_depth=5)


In [36]:
input_cols =['Pclass','Sex','Age','SibSp','Parch','Fare']
output_cols = ['Survived']

In [37]:
sk_tree.fit(train_data[input_cols],train_data[output_cols])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [38]:
sk_tree.predict(test_data[input_cols])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0])

In [39]:
sk_tree.score(test_data[input_cols],test_data[output_cols])

0.8283582089552238

In [40]:
import sys
!{sys.executable} -m pip install graphviz.executables

ERROR: Could not find a version that satisfies the requirement graphviz.executables (from versions: none)
ERROR: No matching distribution found for graphviz.executables
You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
import pydotplus

from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

In [42]:
dot_data = StringIO()
export_graphviz(sk_tree,out_file=dot_data,filled = True, rounded = True)

In [43]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


In [44]:
Image(graph.create())

InvocationException: GraphViz's executables not found